## SEARCHING FOR WATERBODIES IN THE CALAKMUL BIOSPHERE RESERVE

 Jugal Patel<sup>A</sup>  
 Margaret Kalacska<sup>A</sup>, Raja Sengupta<sup>A</sup>, Rafael Reyna<sup>B</sup>

 <sup>A</sup> *Department of Geography, McGill University, Montreal, Quebec*  
 <sup>B</sup> *Departamento de Conservación de la Biodiversidad, El Colegio de la Frontera Sur, Campeche, Mexico*
 
---

### 1. CONTEXT
---

#### 1.1 CALAKMUL BIOSPHERE RESERVE

The Calakmul Biosphere Reserve (CBR) is the largest protected tropical forest in Mexico with ~ 7230 km<sup>2</sup>  host to considerable floral and faunal diversity (Garcia-Gil et al., 1999; Morales and Magana 2001; Pinzon et al., 2021). 

![calakmul](jp_data/calakmul_resize.png)

<center>FIGURE 1: CALAKMUL BIOSPHERE RESERVE</center>  
<center>CAMPECHE, MEXICO</center>  

Protected in 1989, the CBR stands to lose its conservation value as climate change and increasing drought stress push animals, most notably groups of white-lipped and collared peccaries (*Tayassu pecari* and *Pecari tajacu*), south into more humid environments (Reyna-Hurtado et al., 2012; O'Farril et al., 2014; Pinzon et al., 2021). Peccary groups, both white-lipped and collared, are particularly well adapted to the spatial density and distribution of waterbodies. for 

And, as species best associated with waterbodies move south away from these regions in search of water, other animals relying on peccaries for sustenance will also be forced to migrate south. 

#### 1.2 WATERBODIES

While no permanent waterbodies persist in the CBR, intermittent waterbodies -- known locally as 'sartenejas', 'aguadas', and 'civales' -- become apparent based on precipitation and environmental conditions. Sartenejas are very small, often < 1 metre in diameter, depressions on limestone which fill with low amounts of precipitation (Reyna, 20xx). Aguadas are 3 - 10 metres in diameter, circular, shallow, and fill with precipitation as rainwater follows topographic gradients (Reyna 20xx). Additionally, aguadas are critical sources of water for local human populations. Tweleve aguadas in the Southern zone of the CBR are monitored using camera-traps to determine water level, as well as faunal activity. While civales are small, ephemeral, marshy wetlands with shallow bodies of water within them, they are referred to as a type of intermittent lake by local peoples adajacent to the CBR in Guatemala (Ensley et al., 2021). 


Civales ...   
are ... distinguished here as > 10 m ...   

The relative availability of water at these waterbodies is not linear; sartenejas are maintain even when aguadas run dry 

Depending on context (i.e., socio-ecological or biological conditions) fauna in the forest reserve rely on one, two, or all three of these intermittent waterbodies. Sartenejas, aguadas, and civales each fill according to ...   

As well, streams fill aguadas and civales ...  
... check bernard lab dataset? 

The changing spatial distribution of waterbodies in the region are a proxy for understanding drought stress // worsening drought // climate change // ...

### 2. DATA
---

#### 2.1 PLANET

Planet, founded in 2010 as Cosmolgia Inc., is a publicly traded (NYSE:PL) public benefit corporation with the stated mission  "to accelerate humanity to a more sustainable, secure and prosperous world by illuminating environmental and social change." However, characterizing environmental changeat computational and global scales requires data velocity as well as well either methods for data 

Planet offers high and medium resolution imagery
imaging the entire global daily ... (cite: variability study)  
... these offer potential for characterisation of the Earth's land surface ...  
... talk about satelitte consetellation, weird tiling etc. 

Planetscope, Planet's medium resolution imagery is ...  
... collected at a spatial resolution of 3.7 m and resampled to 3 x 3 metre pixels in availed imagery ... 

4 band (https://developers.planet.com/docs/data/psscene4band/)  
sensor information (https://developers.planet.com/docs/apis/data/sensors/)

Planet API ... offers ... 

Filters ... 

NOTE: ... all planetscope data have been repackaged as a single planetscope ... (https://www.planet.com/pulse/planet-launches-next-generation-planetscope-with-eight-spectral-bands-and-quality-improvements/) 

In [ ]:
# libraries
import os
import sys
import json
import time
import requests
import urllib.request
from datetime import datetime
from requests.auth import HTTPBasicAuth

In [ ]:
# variable definition
# output folder name
# wet months
# scene_folder_name = '201701'
# scene_folder_name = '201702'

# dry months
scene_folder_name = '201709'
# scene_folder_name = '201708'
# scene_folder_name = '201707'

# set up session
item_type = "PSScene"
item_types = ["PSScene"]
URL = "https://api.planet.com/data/v1"
os.environ['PL_API_KEY'] = 'xxxx' # enter api key here
PLANET_API_KEY = os.getenv('PL_API_KEY')
session = requests.Session()
session.auth = (PLANET_API_KEY, "")
res = session.get(URL)
quick_url = "{}/quick-search".format(URL) # setup the quick search endpoint url

print("downloading scenes for: " + scene_folder_name)

# setup folder structure
# data_folder = r'data/scenes/' + scene_folder_name
data_folder = r'D:/Users/jpatel27/ponds_wlp/data/scenes/' + scene_folder_name

if not os.path.exists(data_folder):
    os.makedirs(data_folder)  # create folder if it does not exist

In [ ]:
# data download for ortho_analytic_4b_sr
with open('D:/Users/jpatel27/ponds_wlp/data/calakmul.geojson', 'r') as j:
    os.chdir(data_folder)

    # Read AOI as geojson
    geojson_geometry = json.loads(j.read())

    # ---------------------------filters---------------------------#
    # get images that overlap with our AOI
    geometry_filter = {"type": "GeometryFilter","field_name": "geometry","config": geojson_geometry}

    date_range_filter = {"type": "DateRangeFilter","field_name": "acquired","config": {
        "gte": "2017-09-01T00:00:00.000Z","lte": "2017-09-30T00:00:00.000Z"}}
    
    # only get images which have <50% cloud coverage
    cloud_cover_filter = {"type": "RangeFilter","field_name": "cloud_cover","config": {"lte": 0.5}}
    
    # combine our geo, date, cloud filters
    combined_filter = {"type": "AndFilter","config": [geometry_filter, date_range_filter, cloud_cover_filter]}

    # setup the request
    request = {"item_types": item_types,"filter": combined_filter}
    
    # ---------------------------download ---------------------------#

    # Send the POST request to the API quick search endpoint
    res = session.post(quick_url, json=request)
    # Assign the response to a variable
    geojson = res.json()
    # extract image IDs only
    image_ids = [feature['id'] for feature in geojson['features']]
    print(image_ids)

    # all available assets for image id
    for i in range(len(image_ids)):
        print("current scene:" , image_ids[i])
        id0 = image_ids[i]
        id0_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, id0)

        # Returns JSON metadata for assets in this ID. Learn more: planet.com/docs/reference/data-api/items-assets/#asset
        result = \
            requests.get(
                id0_url,
                auth=HTTPBasicAuth(PLANET_API_KEY, '')
            )

        # List of asset types available for this particular satellite image
        result_keys = (result.json().keys())
        # print(result_keys)

        if "ortho_analytic_4b_sr" in result.json():
            # check asset status
            status = result.json()['ortho_analytic_4b_sr']['status']

            # activate asset
            # Parse out useful links
            # insert product here
            links = result.json()[u"ortho_analytic_4b_sr"]["_links"]
            self_link = links["_self"]
            activation_link = links["activate"]

            # Request activation of the 'analytic' asset:
            activate_result = \
                requests.get(
                    activation_link,
                    auth=HTTPBasicAuth(PLANET_API_KEY, '')
                )
            # assign inital status to cur_activation_status
            cur_activation_status = status
            print("current status ", datetime.now(), " ", status)
            # if status is already active
            # comment out if you only need inactive scenes
            if status == 'active':
                # retrieve download link
                activation_status_result = \
                    requests.get(
                        self_link,
                        auth=HTTPBasicAuth(PLANET_API_KEY, '')
                    )
                download_link = activation_status_result.json()["location"]
                # print(download_link)
                urllib.request.urlretrieve(download_link, image_ids[i])
                print("download complete")
                sys.stdout.flush()

            # if asset is not yet active
            if status != 'active':
                while cur_activation_status != 'active':
                    # poll every 30 seconds
                    time.sleep(30)
                    activation_status_result = \
                        requests.get(
                            self_link,
                            auth=HTTPBasicAuth(PLANET_API_KEY, '')
                        )
                    # update cur_activation_status to current status
                    cur_activation_status = activation_status_result.json()["status"]
                    print("current status ", datetime.now(), " ", cur_activation_status)
                    # if asset is activated
                    if cur_activation_status == 'active':
                        # retrieve download link
                        download_link = activation_status_result.json()["location"]
                        urllib.request.urlretrieve(download_link, image_ids[i])
                        print("download complete: ", download_link)
                        continue
        else:
            print(image_ids[i], ' n/a')

#### 2.2 MOSAIC

For both seasons, three months were composited to create mosaics of the study area ...   

The dry season mosaic is a composite of all images collected by (the planetscope sensor) of the Calakmul Biosphere Reserve from December 2016 to February 2017 ...

(insert dry mosaics png) 

the wet season mosaic is a composite of all images collected by (xyz) in our study area from July 2017 - September 2017. 

(insert wet mosaics png) 

To create the above mosaics, the ENVI 5.6 *Seamless Mosaic.* tool was used. To run this tool, each image (also referred to as 'Planet Scene(s)' or simply 'scene(s)' throughout) must be georeferenced and be the same data type. While wavelengths may vary, the number of bands across images must be consistent. In data with variable wavelength (i.e., collected with different sensors or calibration), spectral subsets must be created to ensure image compatibility. 

In our mosaics, all prior images contained four bands with consistent wavelengths for Blue, Green, Red and Near Infra-Red (NIR) and were formatted by Planet as GeoTIFF - georeferenced TIFF files.

![mosaic_all](jp_data/mosaic_all.png)

#### 2.3  IMAGE QUALITY ISSUES

Mosaics of the CBR were not used as   
... they were not complete (for wet season, an important area over known aguadas is absent
... they remain cloudy 
... methods in ENVI 5.6 do not scale without ENVI-PY, and these methods remain poorly documented. 

#### 2.3.1  *CLOUD MASK*

A cloud mask was attempted cross a subset image using example-based classification. 
![cloud mask](jp_data/cloud_mask.png)

### 3. METHODS
---

#### 3.1 CLEAR MOSAIC 

Based on data quality issues, i chose to work with clear imagery where possible ... 

By visually ...

True Color Composite             |  False Colour Composite
:-------------------------:|:-------------------------:
![clear mosaic](jp_data/roi_mosaic.png) | ![clear mosaic](jp_data/roi_mosaic_nir.png)



#### 3.2 INDICES

Indices capture ...  

To use indices, images must include wavelength, wavelength units, and full-width-half-mazimum metadata. This information for planetscope bands is available ... 

#### 3.2.1 *NORMALISED DIFFERENCE VEGETATION INDEX*

NDVI ... 

#### 3.2.2 *SOIL ADJUSTED VEGETATION INDEX*

SAVI / OSAVI ... WAS USED TO MASK 

![ndvi](jp_data/rsz_ndvi_savi.png)

#### 3.2.3 *NORMALISED DIFFERENCE WATER INDEX*

NDWI ... 

#### 3.2.4 *NORMALISED DIFFERENCE LAKE INDEX*

NDLI ... 

![ndvi](jp_data/rsz_ndli_ndwi.png)

#### 3.3 RULE-BASED CLASSIFICATION

to classify pixels based on specified spectra ... 

### 4. RESULTS
---

#### 3.2.4 *TRAINING*

NDLI ... 

#### 4.1 RULE-BASED CLASSIFICATION

Rule-based feature extraction is a tool ENVI provides for classifying ... 
parameters
- threshold

In [ ]:
## output  histogram of area, histogram for roundness 

#### 4.1 SPATIAL LOGIC FOR WATERBODIES

OF the 28k observed polygons which matched specified spectral characteristics, and given the lack of sufficient training data 


came up with 'these' filters for x, y, and z

In [22]:
# basemap options for folium maps
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
}

In [23]:
import folium 

# function to create dynamic maps
def map_function(ctr_lat, ctr_lon, geojson_file_name,zoom_start_lvl= 14, zoom_scroll=False, color = 'NA'):
    file = open(geojson_file_name, encoding="utf8")
    text = file.read()
    m = folium.Map(location=[ctr_lat, ctr_lon], 
                   zoom_start=zoom_start_lvl,
                   zoom_control=zoom_scroll,
                   scrollWheelZoom=zoom_scroll,
                   tiles = None
                   ) # study area scene 
    if color == 'NA':
        folium.GeoJson(text, name="Lakes Classified").add_to(m)
    else:
        style1 = {'fillColor': color,'color': color}
        folium.GeoJson(text, name="Lakes Classified",style_function=lambda x:style1).add_to(m)
    # Add custom basemaps
    basemaps['Google Terrain'].add_to(m)
    basemaps['Google Satellite Hybrid'].add_to(m)
    m.add_child(folium.LayerControl(collapsed = False))
    return m

In [24]:
map_function(18.231862185818112, 
             -89.959829903259021, 
             r"jp_data/rb_class_lakes_filtered_fixed_lessfields.geojson")

### 5. VALIDATION
---

In [28]:
map_function(18.398696553867047, -89.951932307343966 , 
             geojson_file_name = r"jp_data/rb_class_lakes_filtered_select5.geojson", 
             zoom_start_lvl = 22, 
             zoom_scroll = True,
            color = '#fff700')

In [27]:
map_function(18.397445920347188, -89.990709032582089,
             geojson_file_name = r"jp_data/rb_class_lakes_filtered_select5.geojson", 
             zoom_start_lvl = 22, 
             zoom_scroll = True,
            color = '#fff700')

In [19]:
map_function(18.348591542006542,-89.914949593306204,
             geojson_file_name = r"jp_data/rb_class_lakes_filtered_select5.geojson", 
             zoom_start_lvl = 22, 
             zoom_scroll = True,
            color = '#fff700')

In [20]:
map_function(18.185718568458661,-89.995582408685834,
             geojson_file_name = r"jp_data/rb_class_lakes_filtered_select5.geojson", 
             zoom_start_lvl = 22, 
             zoom_scroll = True,
            color = '#fff700')

In [29]:
map_function(18.184093021137567,-89.993825061377763,
             geojson_file_name = r"jp_data/rb_class_lakes_filtered_select5.geojson", 
             zoom_start_lvl = 22, 
             zoom_scroll = True,
            color = '#fff700')

### 6. DISCUSSION
---

#### 6.1 PLANET

Planet images were underwhelming given the spatial extent of CBR, inconsistent image orientations, and the lack of cloud mask products for the time high cloud cover in the region ... 

#### 6.2 FUTURE DIRECTIONS

Apply classification model  
... at scale, with an automated scene order set operation; 
... with GEE if the above is not straightforward in ENVI 
... with differentiation for wet and dry seasons  
... is roundness appropriate given pixels are ...

Collect 2019 imagery and   
... use Planet's UDM2 cloud mask

### 7. CONCLUSION
---

this exploratory study shows promise for using planet imagery... 

### 8. REFERENCES
---

Garcia-Gil et al., 1999
Pinzon et al., 2021
Reyna-Hurtado et al., 2010
Reyna-Hurtado et al., 2012
Reyna-Hurtado et al., 2014
Reyna-Hurtado et al., 2017
